<a href="https://colab.research.google.com/github/albertponfe/Projects/blob/main/RQ2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import warnings

In [10]:
from google.colab import files
import pandas as pd

# Upload the file if it's not already in your Colab environment
files.upload() # Uncomment this line and run to upload the file

statewise_pm25 = pd.read_csv('StatewisePM2.5Concentrations2016-2020.csv')
statewise_pm25.head()

Saving StatewisePM2.5Concentrations2016-2020.csv to StatewisePM2.5Concentrations2016-2020.csv


,statefips,year,avg_ds_pm_pred
0,1,2016,9.725386
1,1,2017,9.318753
2,1,2018,8.911884
3,1,2019,9.167352
4,1,2020,8.730336


In [11]:
print("\n[Step 1] Loading COPD data from CDC Chronic Disease Indicators API...")

# API endpoint for CDC Chronic Disease Indicators
base_url = "https://data.cdc.gov/resource/hksd-2xuw.json"

# Query parameters to filter for COPD data
# We'll fetch a large dataset and filter locally
params = {
    "$limit": 50000,  # Get large dataset
    "topic": "Chronic Obstructive Pulmonary Disease"
}

try:
    response = requests.get(base_url, params=params)
    response.raise_for_status()
    data = response.json()
    df_copd = pd.DataFrame(data)
    print(f"✓ Successfully loaded {len(df_copd)} COPD records")
except Exception as e:
    print(f"✗ Error loading data: {e}")
    print("\nFalling back to sample query...")
    # If the filtered query fails, try without topic filter
    response = requests.get(base_url, params={"$limit": 50000})
    data = response.json()
    df_all = pd.DataFrame(data)
    # Filter for COPD manually
    df_copd = df_all[df_all['topic'] == 'Chronic Obstructive Pulmonary Disease'].copy()
    print(f"✓ Successfully filtered to {len(df_copd)} COPD records")



[Step 1] Loading COPD data from CDC Chronic Disease Indicators API...
✓ Successfully loaded 26951 COPD records


In [12]:
# Convert data value to numeric
df_copd['datavalue'] = pd.to_numeric(df_copd['datavalue'], errors='coerce')

In [13]:
df_copd['yearstart'] = pd.to_numeric(df_copd['yearstart'], errors='coerce')

In [14]:
df_2020 = df_copd[df_copd['yearstart']==2020]

In [15]:
df_2020 = df_2020[df_2020['datavaluetype']=='Age-adjusted Prevalence']

In [17]:
df_2020 =  df_2020[df_2020['locationdesc'].str.contains('Puerto Rico')==False]

In [18]:
df_2020_overall_adults = df_2020[df_2020['question']=='Chronic obstructive pulmonary disease among adults']

In [19]:
df_2020_overall_adults = df_2020_overall_adults[df_2020_overall_adults['stratification1']=='Overall']

In [20]:
states_to_use = ['California', 'Florida', 'New York', 'Oregon', 'Texas']
df_2020_states = df_2020_overall_adults[df_2020_overall_adults['locationdesc'].isin(states_to_use)]

In [21]:
df_2020_overall_adults.columns

Index(['yearstart', 'yearend', 'locationabbr', 'locationdesc', 'datasource',
       'topic', 'question', 'datavalueunit', 'datavaluetype', 'datavalue',
       'datavaluealt', 'lowconfidencelimit', 'highconfidencelimit',
       'stratificationcategory1', 'stratification1', 'geolocation',
       'locationid', 'topicid', 'questionid', 'datavaluetypeid',
       'stratificationcategoryid1', 'stratificationid1',
       'datavaluefootnotesymbol', 'datavaluefootnote'],
      dtype='object')

In [22]:
copd2020 = df_2020_overall_adults[['yearstart', 'locationdesc', 'datavaluetype', 'datavalue', 'stratification1']]

In [23]:
us_state_fips = {
    'Alabama': 1,
    'Alaska': 2,
    'Arizona': 4,
    'Arkansas': 5,
    'California': 6,
    'Colorado': 8,
    'Connecticut': 9,
    'Delaware': 10,
    'District of Columbia': 11,
    'Florida': 12,
    'Georgia': 13,
    'Hawaii': 15,
    'Idaho': 16,
    'Illinois': 17,
    'Indiana': 18,
    'Iowa': 19,
    'Kansas': 20,
    'Kentucky': 21,
    'Louisiana': 22,
    'Maine': 23,
    'Maryland': 24,
    'Massachusetts': 25,
    'Michigan': 26,
    'Minnesota': 27,
    'Mississippi': 28,
    'Missouri': 29,
    'Montana': 30,
    'Nebraska': 31,
    'Nevada': 32,
    'New Hampshire': 33,
    'New Jersey': 34,
    'New Mexico': 35,
    'New York': 36,
    'North Carolina': 37,
    'North Dakota': 38,
    'Ohio': 39,
    'Oklahoma': 40,
    'Oregon': 41,
    'Pennsylvania': 42,
    'Rhode Island': 44,
    'South Carolina': 45,
    'South Dakota': 46,
    'Tennessee': 47,
    'Texas': 48,
    'Utah': 49,
    'Vermont': 50,
    'Virginia': 51,
    'Washington': 53,
    'West Virginia': 54,
    'Wisconsin': 55,
    'Wyoming': 56
}

In [24]:
copd2020['State FIPS'] = copd2020['locationdesc'].map(us_state_fips)

/tmp/ipython-input-4241083572.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  copd2020['State FIPS'] = copd2020['locationdesc'].map(us_state_fips)


In [25]:
copd2020 = copd2020.dropna()
copd2020['State FIPS'] = copd2020['State FIPS'].astype(int)

In [26]:
statepm25_2020 = statewise_pm25[statewise_pm25['year']==2020]

In [27]:
quartiles = np.quantile(statepm25_2020['avg_ds_pm_pred'], [0.25, 0.5, 0.75])
quartiles

array([6.83211735, 7.63903427, 8.29546089])

In [28]:
statepm25_2020['above 75th percentile'] = statepm25_2020['avg_ds_pm_pred'] >= quartiles[2]

/tmp/ipython-input-1580175528.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  statepm25_2020['above 75th percentile'] = statepm25_2020['avg_ds_pm_pred'] >= quartiles[2]


In [29]:
pm_and_copd = pd.merge(statepm25_2020, copd2020, left_on='statefips', right_on='State FIPS', how='left')

In [30]:

pm_and_copd = pm_and_copd[['statefips', 'year', 'avg_ds_pm_pred', 'above 75th percentile', 'locationdesc', 'datavalue']]

In [35]:
state_income = pd.read_csv('/state_income.csv')

In [36]:
state_income = state_income[state_income['Label (Grouping)']== 'Median income (dollars)']

In [37]:
pivot_income = state_income.transpose()

In [38]:
p_income = pivot_income[pivot_income.index.str.contains("Estimate", na=False)]

In [39]:
p_income.index = p_income.index.str.split("!!").str[0]

In [40]:
p_income.rename(columns={11: 'Median Household Income'}, inplace=True)

/tmp/ipython-input-3041956585.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  p_income.rename(columns={11: 'Median Household Income'}, inplace=True)


In [41]:
pm_copd_inc = pd.merge(pm_and_copd, p_income, left_on='locationdesc', right_index=True, how='left')

In [42]:
pm_copd_inc['Median Household Income'] = pm_copd_inc['Median Household Income'].str.replace(",", "").astype(int)

In [44]:
# 1. Load COPD + Tobacco data from CDC API
base_url = "https://data.cdc.gov/resource/hksd-2xuw.json"
params = {
    "$limit": 100000,
    "$where": "topic in ('Chronic Obstructive Pulmonary Disease','Tobacco')"
}

response = requests.get(base_url, params=params)
response.raise_for_status()
df = pd.DataFrame(response.json())

# Make sure year is numeric
df["yearstart"] = pd.to_numeric(df["yearstart"], errors="coerce")

# 2. Define masks for the two indicators we need (all years)

# Overall current cigarette smoking among adults
smoke_mask = (
    (df["topic"] == "Tobacco") &
    (df["question"] == "Current cigarette smoking among adults") &
    (df["stratification1"] == "Overall") &
    (df["datavaluetype"] == "Age-adjusted Prevalence") &
    (df["yearstart"] == 2020)
)


# 3. Build state-year data and average over years

smoking_all = df[smoke_mask].copy()
smoking_all["datavalue"] = pd.to_numeric(smoking_all["datavalue"], errors="coerce")
smoking_all = smoking_all.rename(columns={"datavalue": "smoking_all_pct"})

In [45]:
smoking_all = smoking_all[['locationdesc', 'smoking_all_pct']]

In [46]:
pm_copd_inc_smo = pd.merge(pm_copd_inc, smoking_all, on='locationdesc', how='left')

In [47]:
pm_copd_inc_smo

,statefips,year,avg_ds_pm_pred,above 75th percentile,locationdesc,datavalue,Median Household Income,smoking_all_pct
0,1,2020,8.730336,True,Alabama,8.9,52035,19.5
1,10,2020,7.212074,False,Delaware,5.2,69110,15.8
2,11,2020,7.043888,False,District of Columbia,4.6,90842,11.7
3,12,2020,8.010794,False,Florida,6.2,57703,15.5
4,13,2020,8.908263,True,Georgia,6.2,61224,16.3
5,16,2020,8.653629,True,Idaho,5.2,58915,14.2
6,17,2020,8.973212,True,Illinois,5.7,68428,12.8
7,18,2020,8.713603,True,Indiana,7.6,58235,20.2
8,19,2020,7.648658,False,Iowa,5.6,61836,16.8
9,20,2020,8.031701,False,Kansas,5.8,61091,17.2


In [49]:
urban_rural = pd.read_csv('/urban and rural 2020.csv')

In [50]:
urban_rural = urban_rural.set_index('Label (Grouping)')

In [51]:
urban_rural_t = urban_rural.transpose()
urban_rural_t = urban_rural_t.iloc[:, [0,1]]

In [52]:
u_r = urban_rural_t.set_axis(['Total', 'Urban'], axis=1)
u_r['Total'] = u_r['Total'].str.replace(",", "").astype(int)
u_r['Urban'] = u_r['Urban'].str.replace(",", "").astype(int)

In [53]:
u_r['Urban pct'] = (u_r['Urban']/u_r['Total'])*100

In [54]:
causal_inference = pd.merge(pm_copd_inc_smo, u_r, left_on='locationdesc', right_index=True, how='left')

In [55]:
causal_inference['above 75th percentile'] = causal_inference['above 75th percentile'].astype(int)
causal_inference['smoking above avg'] = (causal_inference['smoking_all_pct']>19.0).astype(int)

In [56]:
causal_inference

,statefips,year,avg_ds_pm_pred,above 75th percentile,locationdesc,datavalue,Median Household Income,smoking_all_pct,Total,Urban,Urban pct,smoking above avg
0,1,2020,8.730336,1,Alabama,8.9,52035,19.5,2288330,1325120,57.907732,1
1,10,2020,7.212074,0,Delaware,5.2,69110,15.8,448735,378613,84.373405,0
2,11,2020,7.043888,0,District of Columbia,4.6,90842,11.7,350364,350364,100.000000,0
3,12,2020,8.010794,0,Florida,6.2,57703,15.5,9865350,9071943,91.957640,0
4,13,2020,8.908263,1,Georgia,6.2,61224,16.3,4410956,3236998,73.385407,0
5,16,2020,8.653629,1,Idaho,5.2,58915,14.2,751859,502775,66.870916,0
6,17,2020,8.973212,1,Illinois,5.7,68428,12.8,5426429,4685875,86.352830,0
7,18,2020,8.713603,1,Indiana,7.6,58235,20.2,2923175,2104049,71.978209,1
8,19,2020,7.648658,0,Iowa,5.6,61836,16.8,1412789,891330,63.090101,0
9,20,2020,8.031701,0,Kansas,5.8,61091,17.2,1275689,910571,71.378761,0


In [57]:
import statsmodels.api as sm

treatment = 'above 75th percentile'
outcome = 'datavalue'
confounders = ['smoking_all_pct', 'Urban pct', 'Median Household Income']


In [58]:
X_simple = causal_inference[[treatment]]
X_simple = sm.add_constant(X_simple)
y = causal_inference[outcome]

model_simple = sm.OLS(y, X_simple).fit()
print(model_simple.summary())
print(model_simple.params[treatment])

                            OLS Regression Results                            
Dep. Variable:              datavalue   R-squared:                       0.032
Model:                            OLS   Adj. R-squared:                  0.012
Method:                 Least Squares   F-statistic:                     1.567
Date:                Mon, 15 Dec 2025   Prob (F-statistic):              0.217
Time:                        01:12:52   Log-Likelihood:                -94.812
No. Observations:                  49   AIC:                             193.6
Df Residuals:                      47   BIC:                             197.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     5.89

In [59]:
X_full = causal_inference[[treatment] + confounders]
X_full = sm.add_constant(X_full)
y = causal_inference[outcome]

model_full = sm.OLS(y, X_full).fit()
print(model_full.summary())

treatment_effect = model_full.params[treatment]
print(treatment_effect)

                            OLS Regression Results                            
Dep. Variable:              datavalue   R-squared:                       0.694
Model:                            OLS   Adj. R-squared:                  0.667
Method:                 Least Squares   F-statistic:                     24.98
Date:                Mon, 15 Dec 2025   Prob (F-statistic):           7.72e-11
Time:                        01:12:59   Log-Likelihood:                -66.578
No. Observations:                  49   AIC:                             143.2
Df Residuals:                      44   BIC:                             152.6
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                     

In [60]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = causal_inference[confounders]
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(
    X_scaled,
    columns=[f'{col}_scaled' for col in confounders],
    index=causal_inference.index
)
X_scaled_df['above 75th percentile'] = causal_inference['above 75th percentile']


X_with_const = sm.add_constant(X_scaled_df)


y = causal_inference['datavalue']
analysis_data = pd.concat([X_with_const, y], axis=1).dropna()
X_final = analysis_data.drop('datavalue', axis=1)
y_final = analysis_data['datavalue']


# Fit OLS model
ols_model = sm.OLS(y_final, X_final)
results = ols_model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:              datavalue   R-squared:                       0.694
Model:                            OLS   Adj. R-squared:                  0.667
Method:                 Least Squares   F-statistic:                     24.98
Date:                Mon, 15 Dec 2025   Prob (F-statistic):           7.72e-11
Time:                        01:12:59   Log-Likelihood:                -66.578
No. Observations:                  49   AIC:                             143.2
Df Residuals:                      44   BIC:                             152.6
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
const       

In [61]:
treatment_effect = results.params[treatment]
print(treatment_effect)

0.5328861704816505
